In [25]:
from IPython.display import HTML, display
display(HTML('''
 <link rel="stylesheet" href="custom.css" type="text/css" />
'''))

In [26]:
!qr --factory=pymaging http://bit.ly/scipyla_scada2iot > img/qr.png

<center>
[![View this Presentation Online](img/qr.png?refresh)](http://bit.ly/scipyla_scada2iot)
</center>
View this presentation on Notebook Viewer

# From SCADA to IoT with Python

## Nahuel Defossé

<h3>
<a href="https://twitter.com/D3f0"><i class="fa fa-twitter" aria-hidden="true">&nbsp;</i></a>
<a href="https://github.com/D3f0"><i class="fa fa-github" aria-hidden="true">&nbsp;</i></a>
</h3>
<h4>
🇪🇸😀🇬🇧😀🇧🇷😅
</h4>


# About me <i class="fa fa-info-circle" aria-hidden="true">&nbsp;</i>
 * Memmber of Research Group focused on Micronctollers and Netowrks.
 * Chief of Practical Assgnments at Universidad Nacional de la Patagonia
 * Python Advocate. It a good hammer 80% of the time.

# Disclaimer
 * Not a good English speaker but definetly worse at Protuguese.
 * Some material cannot be distributed openly, approach me personaly to get more details.

# Owed credit <i class="fa fa-users" aria-hidden="true">&nbsp;</i>

 *  Ricardo A Lopez
 * Fernando A Tinetti 
 * Lautaro Pecile 
 * van Haaster Diego 
 * Wahler Sebastián

# Where do I come from? <i class="fa fa-location-arrow" aria-hidden="true">&nbsp;</i>


# SCADA <i class="fa fa-tachometer" aria-hidden="true">&nbsp;</i>


## Monitor and control industrial processes


# What everyone thinks about SCADA
<img src="img/scadasystem.img_.png" style="width: 80%">

# What SCDA is really like
<center>
![](img/incredible_machine.gif)
</center>

# Mara

* Time sychronization
* State and event retrieval though POLL
* Commnds
* Designed for networks of 8 bit micro-controllers

# Mara Frame

<pre style="font-size: .7em;">
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┐┌─────┐┌─────┬─────┐
│ SOF │ SEQ │ QTY │ SRC │ DST │ CMD │ CMD ││ ... ││ BCL │ BCH │
│     │     │     │     │     │     │     ││     ││     │     │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┘└─────┘└─────┴─────┘
┌─────┬───────────┬───────────┬─────┬───────────┬───────────┐  
│ QSV │ SV0       │ SV1       │ QDI │ DI0       │ DI1       │  
│     │           │           │     │           │           │  
└─────┴───────────┴───────────┴─────┴───────────┴───────────┘  
┌─────┬───────────┬───────────┬─────┬─────────────────────────┐
│ QAI │ AI0       │ AI1       │ QEV │ EV..                    .
│     │           │           │     │                         │
└─────┴───────────┴───────────┴─────┴─────────────────────────┘
</pre>

# Parsing Mara Frame 

## Construct

```python

```

# Implementing Mara

## Structure: Construct

## Sequence: Twisted

# Human Machine Interface (HMI)

<br/>
## Formulas
## SVG

# IoT

## Classical Definition
* **More devices** than people, producing data.
* Constrained devices and Smart Devices.
* Standards
* **Analytics** giving value to data.

## In our context...

### MQTT
* Publisher/Subuscriber protocol.
* QoS

**CoAP** inherits many concepts from HTTP <br/>
**MQTT** Message Queues, QoS.

# Integration with Smart Devices

## Kivy

* Easy to develop
* Quite complete Python implementation
* Good documentation


In [13]:
!git push origin master

Counting objects: 13, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (11/11), done.
Writing objects: 100% (13/13), 4.18 KiB | 0 bytes/s, done.
Total 13 (delta 0), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   15f0d63..7436f93  master -> master


In [16]:
!git add img && git add -u && git commit -m "Updates $(date)" && git push origin master

[master d721543] Updates Sun May 15 22:18:52 ART 2016
 1 file changed, 9 insertions(+), 9 deletions(-)
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 676 bytes | 0 bytes/s, done.
Total 6 (delta 2), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   902ff1c..d721543  master -> master


In [12]:
%%bash
jupyter-nbconvert --to slides From\ Scada\ To\ IoT.ipynb --post serve
# open From\ Scada\ To\ IoT.slides.html

Process is terminated.


In [6]:
!rm reveal.js

In [8]:
!open custom.css

In [24]:
!open .